In [4]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

l = []
o = {}
specs_arr = []
specs_obj = {}

target_url = "https://www.amazon.com/dp/B0BSHF7WHW"

headers = {
    "accept-language": "en-US,en;q=0.9",
    "accept-encoding": "gzip, deflate, br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36",
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7"
}

resp = requests.get(target_url, headers=headers)
print(resp.status_code)
if resp.status_code != 200:
    print(resp)
    exit()

soup = BeautifulSoup(resp.text, 'html.parser')

try:
    o["title"] = soup.find('span', {'id': 'productTitle'}).get_text(strip=True)
except:
    o["title"] = None

images = re.findall('"hiRes":"(.+?)"', resp.text)
o["images"] = images

try:
    price = soup.find("span", {"id": "priceblock_ourprice"})
    if price is None:
        price = soup.find("span", {"id": "priceblock_dealprice"})
    if price is not None:
        o["price"] = price.get_text(strip=True)
    else:
        o["price"] = None
except:
    o["price"] = None

try:
    rating = soup.find("span", {"class": "a-icon-alt"})
    if rating is not None:
        o["rating"] = rating.get_text(strip=True)
    else:
        o["rating"] = None
except:
    o["rating"] = None

specs = soup.find_all("tr", {"class": "a-spacing-small"})

for spec in specs:
    spanTags = spec.find_all("span")
    if len(spanTags) == 2:
        specs_obj[spanTags[0].text] = spanTags[1].text

specs_arr.append(specs_obj)
o["specs"] = specs_arr
l.append(o)

# Create a DataFrame from the list of product information
df = pd.DataFrame(l)

# Save the DataFrame to a CSV file
df.to_csv('amazon_product_info.csv', index=False)
print("Data has been saved to amazon_product_info.csv")


200
Data has been saved to amazon_product_info.csv
